# Import các thư viện và load data

Notebook <br>
Họ và tên: Trần Ngọc Hướng <br>
Mssv: 19021297 <br>
Học phần Học máy <br>

Import thư viện

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib as mp 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

Loading data

In [ ]:
train_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
print("train_data",train_data.shape)
print("test_data",test_data.shape)

# Phân tích dữ liệu

In ra một vài dữ liệu ở tập huấn luyện

In [ ]:
train_data.head(5)

In [ ]:
train_data.target.value_counts()

Biểu đồ target 0 và 1

In [ ]:
sns.countplot(data=train_data, x='target')

In ra một vài dữ liệu ở tập train

In [ ]:
test_data.head(5)

In ra một vài câu hỏi chân thành và không chân thành

In [ ]:
print(train_data['question_text'][(train_data['target']==0)].sample(5).values)

In [ ]:
print(train_data['question_text'][(train_data['target']==1)].sample(5).values)

Vẽ word cloud trực quan hóa dữ liệu

In [ ]:
print("Các từ thường xuất hiện trong những câu hỏi không chân thành")
insincere_ques_text =" ".join(train_data[train_data["target"] == 1]["question_text"])
# Create the wordcloud object
wordcloud = WordCloud(width=600, height=600, margin=0).generate(insincere_ques_text)
# Display the generated image:
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
ax.margins(x = 0, y = 0)
plt.show() 

### Xử lý dữ liệu

In [ ]:
import nltk
import sys
import spacy
import string
from unidecode import unidecode
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
#Hàm chuyển chữ viết tắt thành chữ thường
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
#nguồn: https://www.kaggle.com/canming/ensemble-mean-iii-64-36
#Hàm clean tag toán học (math) và URL
def clean_tag(text):
    if '[math]' in text:
        text = re.sub('\[math\].*?math\]', '[formula]', text)
    if 'http' in text or 'www' in text:
        text = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '[url]', text)
    return text

In [ ]:
#Dấu và các ký tự đặc biệt
puncts=[',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

In [ ]:
def clean_punct(x):
    emptyString=""
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, emptyString)
    return x

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
#Không xóa stop word 
def clean_text(text):        
    # chuyển về dạng chữ thường
    text = text.lower()       
    #Clean tag (math,URL)
    text = clean_tag(text)
    # Chuyển các từ viết tắt trong từ điển về dạng thường
    text = clean_contractions(text, contraction_mapping)
    #Xóa dấu và ký tự đặc biệt
    text = clean_punct(text)
    tokens = word_tokenize(text)
    # Bỏ stop word  
    #tokens_not_sw = [word for word in tokens if not word in stopwords]  
    
    # chuyển từ số nhiều về dạng thường
    #text = [lemmatizer.lemmatize(word) for word in tokens_not_sw ] 
    text = [lemmatizer.lemmatize(word) for word in tokens ] 
    text = " ".join(text)

    return text

Kiểm tra các các câu hỏi khi đã clear

In [ ]:
question_sample = train_data.question_text.sample(1).values[0]
question_sample

In [ ]:
clean_text(question_sample)

Làm sạch dữ liệu trên tập train và tập public test

In [ ]:
train_data['clean_text'] = train_data['question_text'].apply(clean_text)
test_data['clean_text'] = test_data['question_text'].apply(clean_text)

In ra một vài data của tập train khi đã làm sạch

In [ ]:
train_data.head(5)

In [ ]:
#train_text = train_data['question_text']
#test_text = test_data['question_text']
train_text = train_data['clean_text']
test_text = test_data['clean_text']
train_target = train_data['target']
all_text = train_text.append(test_text)

# Huấn luyện mô hình với các model

### import các thư viện

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

### Model SVC và CountVector

Sử dụng count vectorizer với ngram_range = (1-2). <br>
Sử dụng pipeline count vectorizer và SVC model

In [ ]:
count_vectorizer = CountVectorizer(analyzer="word", ngram_range=(1,2))
SVC_model = LinearSVC(C=1, random_state=1)
vector_svc_model = Pipeline([('count_vectorizer', count_vectorizer),('SVC', SVC_model)])

In [ ]:
#X = train_data['question_text']
X = train_data['clean_text']
y = train_data['target']

Chia dữ liệu huấn luyện thành 2 phần: 80% để huấn luyện và 20% để đánh giá, tính f1_score

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [ ]:
vector_svc_model.fit(train_X, train_y)

In [ ]:
svc_predictions = vector_svc_model.predict(test_X)
accuracy_score(test_y, svc_predictions)
f1_score(test_y, svc_predictions)

## Logistic regression và count vectorizer

Sử dụng pipeline logistic regression với count vectorizer

In [ ]:
logit_model = LogisticRegression(C=1, random_state=0)
vector_logit_model = Pipeline([('count_vectorizer', count_vectorizer),('logit', logit_model)])

In [ ]:
vector_logit_model.fit(train_X, train_y)

In [ ]:
logit_predictions = vector_logit_model.predict(test_X)
accuracy_score(test_y, logit_predictions)
f1_score(test_y, logit_predictions)

So sánh mô hình SVC và logistic regression 

In [ ]:
print("Mo hinh SVC")
print(classification_report(test_y, svc_predictions))
print( "f1_score = ", f1_score(test_y, svc_predictions))
print("Mo hinh logistic regression")
print(classification_report(test_y, logit_predictions))
print("f1_score = ", f1_score(test_y, logit_predictions))

## Logistic regression vs Tfidf 

### TFIDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_vectorizer.fit(all_text)

count_vectorizer.fit(all_text)

train_text_features_cv = count_vectorizer.transform(train_text)
test_text_features_cv = count_vectorizer.transform(test_text)

train_text_features_tf = tfidf_vectorizer.transform(train_text)
test_text_features_tf = tfidf_vectorizer.transform(test_text)

In [ ]:
train_text.head()

### K-Fold Cross Validation

Chia training data thành 5 fold, sử dụng id-idf và train với mô hình hồi quy logistic

In [ ]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 1000)
test_preds = 0
oof_preds = np.zeros([train_data.shape[0],])

for i, (train_idx,valid_idx) in enumerate(kfold.split(train_data)):
    x_train, x_valid = train_text_features_tf[train_idx,:], train_text_features_tf[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    logit_model = LogisticRegression()
    print('fitting.......')
    logit_model.fit(x_train,y_train)
    print('predicting......')
    print('\n')
    #lưu kết quả vào oof_preds
    oof_preds[valid_idx] = logit_model.predict_proba(x_valid)[:,1]
    test_preds += 0.2*logit_model.predict_proba(test_text_features_tf)[:,1]

Chọn threshold  có f1_score lớn nhất

In [ ]:
for (i) in range(20,30, 1):
    pred_train = (oof_preds > i / 100).astype(np.int)
    print("threshold", i/100, "f1_score", f1_score(train_target, pred_train))

In [ ]:
threshold = 0.22
pred_train = (oof_preds > .22).astype(np.int)
f1_score(train_target, pred_train)

# Submit

In [ ]:
submission = pd.DataFrame.from_dict({'qid': test_data['qid']})
submission['prediction'] = (test_preds>threshold).astype(np.int)
submission.to_csv('submission.csv', index=False)
submission['prediction'] = (test_preds>threshold)